In [1]:
import pandas as pd
import Levenshtein
import math

In [2]:
pd.options.display.max_columns = 22
pd.options.display.max_rows = 400

### Read in all taxes data + prelim cleaning

In [3]:
df_taxes_list = []
path = "~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Raw-Data/"
for i in range(2009, 2020):
    df_taxes_list.append(pd.read_csv(path+"AY"+str(i)+ " Real Property_Personal Property.csv"))
df_full_taxes = pd.concat(df_taxes_list, ignore_index=True)
df_full_taxes.head();

/anaconda3/envs/cds/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/envs/cds/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (6,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_full_taxes.shape

(421915, 21)

In [5]:
df_home = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Raw-Data/fixed_home_v02.csv')
df_cdbg = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Raw-Data/fixed_cdbg.csv')

In [6]:
def genStrip():
    return lambda addr: addr.strip()

In [7]:
def genSpacing():
    def test(word):
        if word == "  ":
            return ""
        elif type(word) == float:
            if math.isnan(word):
                return ""
        else:
            return word
    return test

In [8]:
df_full_taxes['FULL_ADDRESS'] = df_full_taxes['HOUSE_NO'].astype(str).apply(genStrip()) + \
' ' + df_full_taxes['STDIRECT'].apply(genSpacing()) + df_full_taxes["STREET_NAM"].apply(genStrip()) + ' ' + df_full_taxes["STREET_TYPE"].astype(str).apply(genStrip())

In [9]:
df_full_taxes.head(1000)

,OWNER_NAME,HOUSE_NO,STDIRECT,STREET_NAM,STREET_TYPE,UNIT,PROP_ZIP,ZONE_CODE,SUBDIVISION,PARCEL_NO,PREV_VAL,CURR_VAL,VALCHGDATE,TAXDISTRIC,HOMEEXEMPT,TOTALACRES,PROPERTY_CLASS,DIG_STRAT,BUSI_ID,NAICS,PERSKEY,FULL_ADDRESS
0,NICHOLS DORRIS WILDER,0,NaN,A C L RAILROAD,NaN,NaN,NaN,C3,COUNTY LINE - ACREE AREA,00104/00001/014,13100,13100,11/12/2008,2,S0,0.00,RESIDENTIAL,3,NaN,NaN,NaN,0 A C L RAILROAD nan
1,WESTON MONTY,0,,A C L RAILROAD,NaN,NaN,NaN,C3,COUNTY LINE - ACREE AREA,00104/00001/015,26300,26300,11/12/2008,2,S0,0.39,COMMERCIAL,3,NaN,NaN,NaN,0 A C L RAILROAD nan
2,NICHOLS JOE,0,,A C L RAILROAD,NaN,NaN,NaN,C3,COUNTY LINE - ACREE AREA,00104/00001/016,990,990,11/12/2008,2,S0,0.25,COMMERCIAL,3,NaN,NaN,NaN,0 A C L RAILROAD nan
3,ISRAEL CAROLYN R ETAL,0,NaN,A C L RAILROAD,NaN,NaN,NaN,AG,MASSEY,00155/00004/013,21400,21400,4/16/2007,1,S0,17.19,RESIDENTIAL,4,NaN,NaN,NaN,0 A C L RAILROAD nan
4,GEER RET,0,NaN,A C L RAILROAD,NaN,NaN,NaN,AG,LIBERTY EXPRESSWAY INDUSTRIAL AREA,00186/00001/006,28600,28600,6/20/2007,1,S0,14.30,RESIDENTIAL,4,NaN,NaN,NaN,0 A C L RAILROAD nan
5,WILDER SARAH P & DORRIS,0,NaN,A C L RAILROAD,NaN,NaN,NaN,R4,M. L. WILDER,00104/00002/031,3200,3200,NaN,2,S0,0.21,RESIDENTIAL,3,NaN,NaN,NaN,0 A C L RAILROAD nan
6,WILDER SARAH P & DORRIS,0,NaN,A C L RAILROAD,NaN,NaN,NaN,R4,M. L. WILDER,00104/00002/030,4300,4300,NaN,2,S0,0.21,RESIDENTIAL,3,NaN,NaN,NaN,0 A C L RAILROAD nan
7,HENDERSON PEARL WILDER,0,NaN,A C L RAILROAD,NaN,NaN,NaN,R4,M. L. WILDER,00104/00002/004,3000,3000,3/6/2009,2,S0,0.17,RESIDENTIAL,3,NaN,NaN,NaN,0 A C L RAILROAD nan
8,WILDER SARAH P & DORRIS,0,NaN,A C L RAILROAD,NaN,NaN,NaN,R6,COUNTY LINE - ACREE AREA,00104/00001/013,4000,4000,11/12/2008,2,S0,1.05,RESIDENTIAL,3,NaN,NaN,NaN,0 A C L RAILROAD nan
9,HENDERSON PEARL WILDER,0,NaN,A C L RAILROAD,NaN,NaN,NaN,R4,M. L. WILDER,00104/00002/003,9400,9400,10/31/2008,2,S0,0.22,RESIDENTIAL,3,NaN,NaN,NaN,0 A C L RAILROAD nan


In [10]:
df_house = pd.concat([df_home, df_cdbg], ignore_index = True)
df_house.drop('Unnamed: 0', axis = 1, inplace = True)
s_house = pd.Series(df_house['Address'].unique()) # get unique values for housing proj addresses

In [11]:
s_house.shape

(670,)

### Read in addr junction table list and compare

In [12]:
df_addr = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/ToDatabase/addr_junct_table.csv')

In [13]:
f_addresses = df_addr['Address']
t_addresses = pd.Series(df_full_taxes['FULL_ADDRESS'].unique()) # to compare

In [14]:
t_addresses[~t_addresses.isin(f_addresses)].shape

(16224,)

In [15]:
no_tax = f_addresses[~f_addresses.isin(t_addresses)] # addresses that we "don't" have tax data for

In [16]:
no_tax.head(15)

4           509 W GORDON AVE
5           610 W GORDON AVE
6           700 W GORDON AVE
7           708 W GORDON AVE
23             2632 ERICA CT
30        235 BONNY VIEW AVE
33              1200 AUGUSTA
37          205 S COLLINS ST
46          1705 E BROAD AVE
50    1001 RADIUM SPRINGS RD
56           109 W BROAD AVE
57             110 SHELBY LN
58             112 SHELBY LN
65        252 BONNY VIEW AVE
66        243 BONNY VIEW AVE
Name: Address, dtype: object

In [17]:
def lev_list(address_list, address, i): # will help us find near matches
    temp_list = []
    temp_list_u = []
    for u_address in address_list:
        if Levenshtein.distance(address, u_address.upper()) <= i:
            temp_list.append(address)
            temp_list_u.append(u_address.upper())
    temp_list_series = pd.Series(temp_list)
    temp_list_u_series = pd.Series(temp_list_u)
    return temp_list_u_series

In [18]:
lev_list(t_addresses, "509 W GORDON AVE", 3) # some exploratory checking

0        309 GORDON AVE
1        409 GORDON AVE
2        502 GORDON AVE
3        506 GORDON AVE
4        507 GORDON AVE
5        519 GORDON AVE
6        609 GORDON AVE
7        809 GORDON AVE
8        909 GORDON AVE
9     1409 E GORDON AVE
10      1509 GORDON AVE
11    1707 W GORDON AVE
12    1800 W GORDON AVE
13    2400 W GORDON AVE
14    2105 W GORDON AVE
dtype: object

In [19]:
def gen_list_none(address_list, mismatched, i): # just to find how many addresses are so far off from anything in full
    tmp_list = []
    for address in mismatched:
        if address == "UNKNOWN ADDRESS" or address == "SUPPRESSED ADDRESS":
            continue
        if lev_list(address_list, address, i).shape[0] == 0:
            tmp_list.append(address)
    return pd.Series(tmp_list)

### Need to think about types of joins

#### We're trying to modify our tax data so that any conflicts or typos in tax are overwritten by full

#### But if it is not a typo (valid address), then we want to add it to our address full list

#### might be worth to look into an address standardizer

In [20]:
df_full_taxes[df_full_taxes['STREET_NAM'].str.contains("GORDON")]

,OWNER_NAME,HOUSE_NO,STDIRECT,STREET_NAM,STREET_TYPE,UNIT,PROP_ZIP,ZONE_CODE,SUBDIVISION,PARCEL_NO,PREV_VAL,CURR_VAL,VALCHGDATE,TAXDISTRIC,HOMEEXEMPT,TOTALACRES,PROPERTY_CLASS,DIG_STRAT,BUSI_ID,NAICS,PERSKEY,FULL_ADDRESS
12778,MILLER GREG A SR & CHERYL A,0,,GORDON,AVE,NaN,NaN,M1,GORDON AREA WEST,00240/00015/001,14800,14800,9/6/2007,1,S0,7.40,COMMERCIAL,4,NaN,NaN,NaN,0 GORDON AVE
12779,METHODIST CITY BOARD INC,0,NaN,GORDON,AVE,NaN,NaN,C1,GORDON AREA WEST,00240/00002/001,29200,29200,12/26/2007,1,S0,2.92,NaN,2,NaN,NaN,NaN,0 GORDON AVE
12780,D & D WOOD PRESERVING INC,0,,GORDON,AVE,NaN,NaN,M1,GORDON AREA WEST,00240/00014/001,536400,536400,10/31/2008,1,S0,5.28,COMMERCIAL,4,NaN,NaN,NaN,0 GORDON AVE
12781,SOUTH GEORGIA BRICK CO INC,0,,GORDON,AVE,NaN,NaN,M1,GORDON AREA WEST,00240/00014/006,125500,125500,3/11/2009,1,S0,3.05,COMMERCIAL,3,NaN,NaN,NaN,0 GORDON AVE
12782,TRINITY METROPOLITAN BAPTIST CHURCH INC,0,NaN,GORDON,AVE,NaN,NaN,M1,GORDON COMMERCIAL,00240/00009/001,59400,59400,6/16/2009,1,S0,11.89,NaN,2,NaN,NaN,NaN,0 GORDON AVE
12783,HH&A'S LLC,0,,GORDON,AVE,NaN,NaN,R1B,AVONDALE,0000Q/00048/01B,12600,12600,3/14/2007,1,S0,0.52,COMMERCIAL,3,NaN,NaN,NaN,0 GORDON AVE
12784,STARTER HOME BUILDERS LLC &,0,NaN,GORDON,AVE,NaN,NaN,R6,GORDON AREA WEST,00240/00001/009,20600,20600,12/26/2007,1,S0,3.44,RESIDENTIAL,3,NaN,NaN,NaN,0 GORDON AVE
12785,CITY OF ALBANY,307,NaN,GORDON,AVE,NaN,NaN,R6,CITY OF ALBANY/LOTS,0000E/00026/009,2000,2000,7/13/2007,1,S0,0.08,NaN,1,NaN,NaN,NaN,307 GORDON AVE
12786,CITY OF ALBANY,309,NaN,GORDON,AVE,NaN,NaN,R6,CITY OF ALBANY/LOTS,0000E/00026/008,1900,1900,7/13/2007,1,S0,0.07,NaN,1,NaN,NaN,NaN,309 GORDON AVE
12787,JORDAN JOSEPH L MD PC,310,,GORDON,AVE,NaN,NaN,C1,HOLLYWOOD,0000I/00004/11A,347600,347600,10/31/2008,1,S0,1.69,COMMERCIAL,3,NaN,NaN,NaN,310 GORDON AVE


In [21]:
t_addresses[t_addresses.str.contains("GORDON")]

12207         0 GORDON AVE
12208       307 GORDON AVE
12209       309 GORDON AVE
12210       310 GORDON AVE
12211       311 GORDON AVE
12212       315 GORDON AVE
12213       323 GORDON AVE
12214       325 GORDON AVE
12215       327 GORDON AVE
12216       329 GORDON AVE
12217       403 GORDON AVE
12218       405 GORDON AVE
12219       407 GORDON AVE
12220       409 GORDON AVE
12221       411 GORDON AVE
12222       413 GORDON AVE
12223       415 GORDON AVE
12224       419 GORDON AVE
12225       421 GORDON AVE
12226       423 GORDON AVE
12227       425 GORDON AVE
12228       427 GORDON AVE
12229       502 GORDON AVE
12230       506 GORDON AVE
12231       507 GORDON AVE
12232       511 GORDON AVE
12233       512 GORDON AVE
12234       513 GORDON AVE
12235       515 GORDON AVE
12236       518 GORDON AVE
12237       519 GORDON AVE
12238       520 GORDON AVE
12239       522 GORDON AVE
12240       524 GORDON AVE
12241       526 GORDON AVE
12242       528 GORDON AVE
12243       532 GORDON AVE
1

In [22]:
no_tax.iloc[0:1000].to_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Raw-Data/test/no_tax.csv', header=True, index=False)

In [23]:
df_full_taxes['STDIRECT'].value_counts().index

Index(['  ', 'S ', 'N ', 'E ', 'W ', 'SE', 'NE', 'R '], dtype='object')

In [24]:
df_full_taxes['STDIRECT'][0]

nan

In [25]:
df_full_taxes[df_full_taxes['STREET_NAM'].str.contains("MADISON")]

,OWNER_NAME,HOUSE_NO,STDIRECT,STREET_NAM,STREET_TYPE,UNIT,PROP_ZIP,ZONE_CODE,SUBDIVISION,PARCEL_NO,PREV_VAL,CURR_VAL,VALCHGDATE,TAXDISTRIC,HOMEEXEMPT,TOTALACRES,PROPERTY_CLASS,DIG_STRAT,BUSI_ID,NAICS,PERSKEY,FULL_ADDRESS
19617,MADISON PARK DEVELOPMENT INC,0,S,MADISON,ST,NaN,NaN,R1,MADISON PARK,00012/00003/21E,3500,3500,10/16/2007,1,S0,0.07,RESIDENTIAL,3,NaN,NaN,NaN,0 S MADISON ST
19618,MADISON PARK DEVELOPMENT INC,0,S,MADISON,ST,NaN,NaN,R1,MADISON PARK,00012/00003/21F,3500,3500,10/16/2007,1,S0,0.07,RESIDENTIAL,3,NaN,NaN,NaN,0 S MADISON ST
19619,MADISON PARK DEVELOPMENT INC,0,S,MADISON,ST,NaN,NaN,R1,MADISON PARK,00012/00003/21A,3500,3500,10/16/2007,1,S0,0.07,RESIDENTIAL,3,NaN,NaN,NaN,0 S MADISON ST
19620,MADISON PARK DEVELOPMENT INC,0,S,MADISON,ST,NaN,NaN,R1,MADISON PARK,00012/00003/21D,7500,7500,10/1/2008,1,S0,0.16,RESIDENTIAL,3,NaN,NaN,NaN,0 S MADISON ST
19621,ALBANY GA CITY OF DCED,0,S,MADISON,ST,NaN,NaN,R3,RIVER ROAD,00012/00003/20B,730,730,10/16/2007,1,S0,0.23,NaN,1,NaN,NaN,NaN,0 S MADISON ST
19622,THOMAS ROBERT D,0,S,MADISON,ST,NaN,NaN,R3,RIVER ROAD,00012/00002/52C,1700,1700,3/14/2007,1,S0,0.42,RESIDENTIAL,3,NaN,NaN,NaN,0 S MADISON ST
19623,ALBANY GA CITY OF DCED,0,S,MADISON,ST,NaN,NaN,R3,RIVER ROAD,00012/00003/20E,730,730,10/16/2007,1,S0,0.23,NaN,1,NaN,NaN,NaN,0 S MADISON ST
19624,ALBANY GA CITY OF DCED,0,S,MADISON,ST,NaN,NaN,R3,RIVER ROAD,00012/00003/20C,730,730,10/16/2007,1,S0,0.23,NaN,1,NaN,NaN,NaN,0 S MADISON ST
19625,ALBANY GA CITY OF DCED,0,S,MADISON,ST,NaN,NaN,R3,RIVER ROAD,00012/00003/20D,830,830,10/16/2007,1,S0,0.32,NaN,1,NaN,NaN,NaN,0 S MADISON ST
19626,THOMAS JOSEPHINE,0,S,MADISON,ST,NaN,NaN,R3,RIVER ROAD,00012/00003/15C,1000,1000,10/16/2007,1,S0,0.13,RESIDENTIAL,3,NaN,NaN,NaN,0 S MADISON ST


In [26]:
f_addresses.head(100)

0           501 S MADISON ST
1            525 LINCOLN AVE
2           609 S JACKSON ST
3            602 S MONROE ST
4           509 W GORDON AVE
5           610 W GORDON AVE
6           700 W GORDON AVE
7           708 W GORDON AVE
8            504 WHITNEY AVE
9            510 WHITNEY AVE
10           423 WHITNEY AVE
11            402 MERCER AVE
12            404 MERCER AVE
13            408 MERCER AVE
14            522 MERCER AVE
15         414 S MCKINLEY ST
16          411 S MADISON ST
17            525 MERCER AVE
18            511 MERCER AVE
19            507 MERCER AVE
20           118 BLAYLOCK ST
21          1221 E BROAD AVE
22         1218 MULBERRY AVE
23             2632 ERICA CT
24          202 N CARROLL ST
25       1706 S JEFFERSON ST
26    2703 POINTE NORTH BLVD
27            2728 MCCALL CT
28       1000 N SLAPPEY BLVD
29           2703 MICHAEL RD
30        235 BONNY VIEW AVE
31            5101 SHILOH CT
32        2312 WILLINGHAM DR
33              1200 AUGUSTA
34        2531